In [14]:
import pandas as pd
import nltk

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Importing libraries 
import nltk 
import re 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import pandas as pd 

In [15]:
file = 'F://Environmental Baseline Data//Version 4 - Final//Support files//Appendices mapped with files per Project.xlsx'

df = pd.read_excel(file, sheet_name="new")
df.head()
len(df)
#df.columns

Text = df['Component PDF Name']
print(len(Text))
Text[1]

,Application title short,Component,Component PDF Name,file_name,Component Index
0,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 1: Introduction, Part 2...",A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,1
1,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 5: Physical and Meteoro...",A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,2
2,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 9: Fish and Fish Habita...",A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,3
3,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 11: Wildlife and Wildli...",A92619-13 13 ESA Section 11 - A6F4Q6,4
4,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 12: Species at Risk, Pa...",A92619-14 14 ESA Sections 12 to 16 - A6F4Q7,5


1902

1902


'Table of Content, Part 5: Physical and Meteorological Environment, Part 6: Soil and Soil Productivity, Part 7: Vegetation, Part 8: Water Quality and Quantity'

# Frequency, Bigrams, Trigrams

Ref -> https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/

In [16]:
#nltk.download('stopwords')
def remove_string_special_characters(s): 
      
    # removes special characters with ' ' 
    stripped = re.sub('[^a-zA-z\s]', '', s) 
    stripped = re.sub('_', '', stripped) 
      
    # Change any white space to one space 
    stripped = re.sub('\s+', ' ', stripped) 
      
    # Remove start and end white spaces 
    stripped = stripped.strip() 
    if stripped != '': 
            return stripped.lower() 
          

# Stopword removal  
stop_words = set(stopwords.words('english')) 
your_list = ['and', 'is', 'the', 'table', 'appendix', 'esa', 'environment', 'environmental', 
             'sheet', 'report', 'assessment', 'nan', 'plan', 'protection', 'section', 'effects',
            'project'] 
Text_clean = []
for line in Text:
    line_clean = ""
    line = remove_string_special_characters(str(line))
    for x in str(line).split():
        if (x not in stop_words) and (x not in your_list) and len(x) > 2:
            line_clean = line_clean + " " + str(x)
    Text_clean.append(line_clean.strip())
len(Text_clean)
Text_clean[5:13]


1902

['content part heritage resources part navigation navigation safety part traditional land resource use part socioeconomic elements part accidents malfunctions part part summary socioeconomic part compliance part postconstruction monitoring part conclusion',
 '',
 'alignment package',
 'alignment package',
 'soil soil productivity water quality quantity fish fish habitat wildlife wildlife habitat air emissions acoustic preliminary caribou habitat restoration offset measures preliminary decommissioning abandonment traditional knowledge',
 'executive summary contents',
 'introduction',
 'introduction description consultation relation routing siting']

# Frequency

In [17]:
Big_Text_clean = ""
for line in Text_clean:
    Big_Text_clean = Big_Text_clean + " " +line
    
#len(Big_Text_clean.split())

wordlist = Big_Text_clean.split()
wordfreq = []
for w in Big_Text_clean.split():
    wordfreq.append(wordlist.count(w))
    
len(wordlist)
len(wordfreq)

wordlist_unique = []
wordfreq_unique = []
for i in range(len(wordlist)):
    if wordlist[i] not in wordlist_unique:
        wordlist_unique.append(wordlist[i])
        wordfreq_unique.append(wordfreq[i])
        #print("{} : {}".format(wordlist[i], wordfreq[i]))
len(wordfreq_unique)
len(wordlist_unique)

df_wordlist = pd.DataFrame({'Word' : wordlist_unique,
                            'Freq' : wordfreq_unique})
df_wordlist.head()

df_wordlist_sort = df_wordlist.sort_values('Freq',ascending = False)
df_wordlist_sort.head(50)

10693

10693

688

688

,Word,Freq
0,content,20
1,part,248
2,introduction,116
3,description,43
4,consultation,30


,Word,Freq
229,volume,503
45,alignment,478
150,sheets,446
139,technical,443
6,socioeconomic,272
1,part,248
83,data,238
263,marine,201
17,habitat,171
16,fish,169


In [18]:
df_wordlist_sort.head(100)[47:]

,Word,Freq
21,risk,59
127,proposed,55
211,information,52
169,river,48
80,baseline,48
159,expansion,47
132,mainline,47
227,british,46
228,columbia,46
570,poisson,44


# Classification Based on these Keywords 

In [19]:
land1 = ['soil', 'land', 'ground', 'terrain', 'topography', 'ecozones', 'terrain']
air2 = ['air', 'emission', 'ghg', 'gas', 'greenhouse', 'weather' , 'climate', 'meteorological', 'atmospher']
water3 = ['water', 'fish', 'wetlands', 'navigation', 'marine', 'aqua', 'drain', 'river']
wildlife4 = ['wild', 'fish', 'poisson', 'species', 'habitat', 'acoustic', 'life', 'biophysical']
vegetation5 = ['vegetation', 'wetlands', 'plant', 'soil']
human6 = ['human', 'socio', 'social', 'economic', 'economy' 'occupancy', 'heritage', 'health', 'aesthetics', 
          'employment', 'acoustic', 'traditional', 'navigation', 'resource', 'infrastructure', 'noise', 'rapport']
alignment_sheet7 = ['alignment', 'sheet']
tech8 = ['technical', 'tech']
traditional_knowledge9 = ['first', 'nation', 'traditional', 'engage']

In [20]:
def check_topic_present(topic, text):
    for key in topic:
        if key in text:
            return 1
    return 0

Text = df['Component PDF Name']
print(len(Text))
Text[1]
count_blanks = 0

topics = []
for line in Text:
    line = remove_string_special_characters(str(line)).lower()
    line_topics = []
    
    if check_topic_present(land1, line) == 1:
        line_topics.append('Land')
        
    if check_topic_present(air2, line) == 1:
        line_topics.append('Air')
    
    if check_topic_present(water3, line) == 1:
        line_topics.append('Water')
        
    if check_topic_present(wildlife4, line) == 1:
        line_topics.append('Wildlife')
        
    if check_topic_present(vegetation5, line) == 1:
        line_topics.append('Vegetation')
        
    if check_topic_present(human6, line) == 1:
        line_topics.append('Human')
        
    if check_topic_present(alignment_sheet7, line) == 1:
        line_topics.append('Alignment Sheet')
        
    if check_topic_present(tech8, line) == 1:
        line_topics.append('Technology')

    if check_topic_present(traditional_knowledge9, line) == 1:
        line_topics.append('Traditional Knowledge')
        
    if line_topics == []:
        count_blanks = count_blanks +1 
        
    topics.append(line_topics)

len(topics)
len(Text)
count_blanks

1902


'Table of Content, Part 5: Physical and Meteorological Environment, Part 6: Soil and Soil Productivity, Part 7: Vegetation, Part 8: Water Quality and Quantity'

1902

1902

250

In [21]:
df['topics'] = topics
df.head()

,Application title short,Component,Component PDF Name,file_name,Component Index,topics
0,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 1: Introduction, Part 2...",A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,1,"[Human, Traditional Knowledge]"
1,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 5: Physical and Meteoro...",A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,2,"[Land, Air, Water, Vegetation]"
2,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 9: Fish and Fish Habita...",A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,3,"[Land, Water, Wildlife, Vegetation]"
3,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 11: Wildlife and Wildli...",A92619-13 13 ESA Section 11 - A6F4Q6,4,[Wildlife]
4,Application for 2021 NGTL System Expansion Pro...,ESA,"Table of Content, Part 12: Species at Risk, Pa...",A92619-14 14 ESA Sections 12 to 16 - A6F4Q7,5,"[Air, Wildlife, Human]"


In [22]:
file_name = 'F://Environmental Baseline Data//Version 4 - Final//Support files//Table_Title_Topics_for_Brooke.csv'
df.to_csv(file_name, index = False, encoding='utf-8-sig')

# Classification M2M Dataframe

In [28]:
PDF_File_Name =[]
PDF_Component_Names = []
PDF_Topics = []
count_blanks = 0

for index, row in df.iterrows():
    line = remove_string_special_characters(str(row['Component PDF Name'])).lower()
    count = 0
    
    if check_topic_present(land1, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Land')
        count = 1
    if check_topic_present(air2, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Air')
        count = 1
    if check_topic_present(water3, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Water')
        count = 1
    if check_topic_present(wildlife4, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Wildlife')
        count = 1
    if check_topic_present(vegetation5, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Vegetation')
        count = 1
    if check_topic_present(human6, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Human')
        count = 1
    if check_topic_present(alignment_sheet7, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Alignment Sheet')
        count = 1
    if check_topic_present(tech8, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Technology')
        count = 1
    if check_topic_present(traditional_knowledge9, line) == 1:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('Traditional Knowledge')
        count = 1

#     if check_topic_present(tech8, line) == 1:
#         PDF_File_Name.append(row['file_name'])
#         PDF_Component_Names.append(row['Component PDF Name'])
#         PDF_Topics.append(row['land'])
#         count = 1
    if count == 0:
        PDF_File_Name.append(row['file_name'])
        PDF_Component_Names.append(row['Component PDF Name'])
        PDF_Topics.append('other')
        count_blanks = count_blanks +1 
        
len(PDF_File_Name)
len(PDF_Component_Names)
len(PDF_Topics)

3325

3325

3325

In [29]:
count_blanks 

250

In [30]:
df_labels = pd.DataFrame({'file_name': PDF_File_Name,
                          'PDF_Component_Names' : PDF_Component_Names,
                          'PDF_Topics' : PDF_Topics})
df_labels.head(15)

,file_name,PDF_Component_Names,PDF_Topics
0,A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,"Table of Content, Part 1: Introduction, Part 2...",Human
1,A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,"Table of Content, Part 1: Introduction, Part 2...",Traditional Knowledge
2,A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,"Table of Content, Part 5: Physical and Meteoro...",Land
3,A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,"Table of Content, Part 5: Physical and Meteoro...",Air
4,A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,"Table of Content, Part 5: Physical and Meteoro...",Water
5,A92619-11 11 ESA Sections 5 to 8 - A6F4Q4,"Table of Content, Part 5: Physical and Meteoro...",Vegetation
6,A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,"Table of Content, Part 9: Fish and Fish Habita...",Land
7,A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,"Table of Content, Part 9: Fish and Fish Habita...",Water
8,A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,"Table of Content, Part 9: Fish and Fish Habita...",Wildlife
9,A92619-12 12 ESA Sections 9 to 10 - A6F4Q5,"Table of Content, Part 9: Fish and Fish Habita...",Vegetation


In [33]:
file_name = 'F://Environmental Baseline Data//Version 4 - Final//Support files//Tableau_Table_Title_Topics_for_Brookev2.csv'
df_labels.to_csv(file_name, index = False, encoding='utf-8-sig')

In [32]:
file_name = "C://Users//singvibu//Desktop//GitHub//Environment-and-Socio-Economic-Assessmnets-Project//Brooke's Categorization based on keywords//a.csv"
df_labels.to_csv(file_name, index = False, encoding='utf-8-sig')

In [ ]:
response = urllib.request.urlopen(url)
html = response.read()
text = obo.stripTags(html).lower()
fullwordlist = obo.stripNonAlphaNum(text)
wordlist = obo.removeStopwords(fullwordlist, obo.stopwords)
dictionary = obo.wordListToFreqDict(wordlist)
sorteddict = obo.sortFreqDict(dictionary)

for s in sorteddict: print(str(s))

In [ ]:
# html-to-freq-2.py
import urllib.request, urllib.error, urllib.parse
import obo



In [109]:
# Getting trigrams  
vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(Text_clean)  
features = (vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray()) 
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(Text_clean)
scores = (X2.toarray()) 
print("\n\nScores : \n", scores) 
  




Features : 
 ['abandonment plan traditional', 'aboriginal engagement activities', 'aboriginal engagement landowner', 'aboriginal engagement outcomes', 'aboriginal field study', 'accidents malfunctions section', 'accidents malfunctions unplanned', 'accounts techncial report', 'acid rock drainage', 'acoustic environment preliminary', 'acoustic environment section', 'acoustic environment technical', 'acoustic evaluation bakken', 'act clearance letters', 'activities considered cumulative', 'activities planned mitigation', 'activity watercource wildlife', 'additional undefined drainages', 'air emissions acoustic', 'air greenhouse gas', 'air quality assessment', 'air quality greenhouse', 'air quality modelling', 'air quality noise', 'air quality technical', 'aitken creek section', 'aitken kahta paragon', 'alberta british columbia', 'alberta technical report', 'alces river compressor', 'alignment sheet package', 'alignment sheets environmental', 'alignment sheets gas', 'alignment sheets gat

In [110]:
# Getting top ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
print ("\n\nWords head : \n", words.head(7)) 



Words head : 
                                term        rank
256  environmental alignment sheets  392.205149
329      fisheries british columbia   25.980762
136       columbia technical report   25.980762
121      british columbia technical   25.980762
871     vegetation technical report   25.625859
30          alignment sheet package   23.334524
255   environmental alignment sheet   23.334524


# Bigrams

In [112]:
# Getting trigrams  
vectorizer = CountVectorizer(ngram_range = (2,2)) 
X1 = vectorizer.fit_transform(Text_clean)  
features = (vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray()) 
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(Text_clean)
scores = (X2.toarray()) 
print("\n\nScores : \n", scores) 



Features : 
 ['abandonment assessment', 'abandonment plan', 'aboriginal engagement', 'aboriginal field', 'accidents malfunctions', 'accounts techncial', 'acid rock', 'acoustic assessments', 'acoustic environment', 'acoustic evaluation', 'act clearance', 'activities considered', 'activities planned', 'activity watercource', 'additional undefined', 'air emissions', 'air greenhouse', 'air quality', 'aitken creek', 'aitken kahta', 'alberta british', 'alberta technical', 'alces river', 'alignment sheet', 'alignment sheets', 'along liege', 'along northwest', 'alternative crossing', 'appendix air', 'appendix alternative', 'appendix assessment', 'appendix baseline', 'appendix conceptual', 'appendix field', 'appendix hydrology', 'appendix mitigation', 'appendix pathways', 'appendix predicted', 'appendix project', 'appendix socioeconomic', 'appendix soil', 'appendix watercourse', 'aquatic assessment', 'aquatic resources', 'aquatic technical', 'aquatics summary', 'aquatics technical', 'archaeol

In [113]:
# Getting top ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
print ("\n\nWords head : \n", words.head(7)) 



Words head : 
                        term        rank
256  feasibility assessment  392.205149
329   inspection monitoring   25.980762
136     crossing assessment   25.980762
121   considered cumulative   25.980762
255           fall wildlife   23.334524
30      appendix assessment   23.334524
28             appendix air   21.920310


# Frequency 